In [7]:
from calibrators import *

Import data from csv

In [9]:
data = import_from_csv("data/calibrationData_2024-04-29T12-56-12Z.csv")

In [10]:
aoc = AverageOffsetCalibrator(data)
aoc.calibrate((0, 0))

(-588.4875678281361, -1067.617007912797)

In [11]:
woc = WeightedOffsetCalibrator(data)
woc.calibrate((0, 0))

(-577.7242794795692, -850.1651631419289)